In [1]:
#@title Define if we are on Colab and mount drive { display-mode: "form" }
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
#@title (COLAB ONLY) Code for setting the environment, installation of packages { display-mode: "form" }
%%capture
if IN_COLAB:
  %env PYTHONPATH=
  !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
  !chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
  !./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
  !conda install --channel defaults conda python=3.6 --yes -q
  !conda update --channel defaults --all --yes -q
  import sys
  sys.path.insert(0, "/usr/local/lib/python3.6/site-packages")

  # !pip install fastai pydicom kornia fastcore==1.0.9 --upgrade --quiet
  !conda config --add channels conda-forge
  !conda install -c fastai -c pytorch fastai fastcore=1.0.9 pydicom gdcm kornia scikit-image scikit-learn pandas numpy ipykernel --yes -q

In [ ]:
#@title (COLAB ONLY) Clone GitHub repo { display-mode: "form" }

if IN_COLAB:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  import os
  os.environ['GITHUB_AUTH'] = user + ':' + password
  !git clone https://$GITHUB_AUTH@github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

In [2]:
if IN_COLAB:
  PATH_DRIVER = '/content/gdrive/My Drive/'
  DATA_FOLDER = 'Dataset/'
else:
  PATH_DRIVER = ''
  DATA_FOLDER = 'data/'

In [3]:
import os

from fastai.basics import *
from fastai.medical.imaging import *

In [4]:
PATH_PREFIX = os.path.join(PATH_DRIVER, DATA_FOLDER, '')
raw_folder = PATH_PREFIX + 'DICOMS'
organize_folder = PATH_PREFIX + 'pending_classification'
doubt_folder = PATH_PREFIX + 'doubt'
preprocess_folder = PATH_PREFIX + 'preprocess'

correct_check_DICOM_dict = {
    # 'SeriesDescription': ['RODILLA AP', 'RODILLAS AP'],
    'SeriesDescription': ['RODILLA AP'],
    'BodyPartExamined': ['LOWER LIMB', 'KNEE']
}

doubt_check_DICOM_dict = {
    'SeriesDescription': ['PIERNA AP MESA']
}

metadata_labels_path = os.path.join(DATA_FOLDER, 'metadata_labels.csv')
if os.path.exists(metadata_labels_path):
    metadata_labels = pd.read_csv(metadata_labels_path, index_col=1)
else:
    metadata_labels = None

In [5]:
from utils import organize_folders, generate_template, rename_patient

groups = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9']
subgroup_length = 20
relation_filepath = PATH_PREFIX + 'relation.csv'
filename_prefix = 'IMG_'
copy = True

In [6]:
relation_df = organize_folders(raw_folder, organize_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, filename_prefix=filename_prefix, force_extension='.dcm', copy=copy, metadata_labels=metadata_labels, check_DICOM_dict=correct_check_DICOM_dict, debug=False)

In [7]:
_ = organize_folders(raw_folder, doubt_folder, relation_filepath, reset=True, groups=groups, subgroup_length=subgroup_length, filename_prefix=filename_prefix, force_extension='.dcm', copy=copy, check_DICOM_dict=doubt_check_DICOM_dict, debug=False)

In [7]:
dicom_files = get_files(organize_folder, extensions='.dcm')
rename_patient(dicom_files)

In [8]:
generate_template(organize_folder, groups, subgroup_length, filename_prefix=filename_prefix, excel=True)